### SCD Type - 2

In [0]:
%sql
create or replace table datamodel.default.scdtype2_src
(
  product_id int,
  product_name string,
  product_category string,
  processed_date date
);

In [0]:
%sql
insert into datamodel.default.scdtype2_src
values
(1,'Product1','Category1', current_date()),
(2,'Product2','Category2', current_date()),
(3,'Product3','Category3', current_date());

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
create or replace table datamodel.gold.scdtype2_tbl
(
  product_id int,
  product_name string,
  product_category string,
  processed_date date,
  start_date date,
  end_date date,
  is_current boolean
)

#### Merge

In [0]:
%sql
merge into datamodel.gold.scdtype2_tbl dst
using datamodel.default.scdtype2_src src
on dst.product_id = src.product_id
and dst.is_current = true
when matched and (dst.product_name <> src.product_name 
or dst.product_category <> src.product_category 
or dst.processed_date <> src.processed_date) then
update set
  dst.end_date = current_date(),
  dst.is_current = false

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0


In [0]:
%sql
merge into datamodel.gold.scdtype2_tbl dst
using datamodel.default.scdtype2_src src
on dst.product_id = src.product_id
and dst.is_current = true
when not matched then
insert (product_id, product_name, product_category, processed_date, start_date, end_date, is_current)
values (src.product_id, src.product_name, src.product_category, src.processed_date, current_date(), null, true);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql
select * from datamodel.gold.scdtype2_tbl;

product_id,product_name,product_category,processed_date,start_date,end_date,is_current
1,Product1,Category1,2025-08-11,2025-08-11,null,true
2,Product2,Category2,2025-08-11,2025-08-11,null,true
3,Product3,Category3,2025-08-11,2025-08-11,null,true


In [0]:
%sql
update datamodel.default.scdtype2_src
set product_category = 'Category4'
where product_id = 3;

num_affected_rows
1


In [0]:
%sql
select * from datamodel.gold.scdtype2_tbl;

product_id,product_name,product_category,processed_date,start_date,end_date,is_current
1,Product1,Category1,2025-08-11,2025-08-11,null,true
2,Product2,Category2,2025-08-11,2025-08-11,null,true
3,Product3,Category3,2025-08-11,2025-08-11,2025-08-11,false
3,Product3,Category4,2025-08-11,2025-08-11,null,true
